# TP 3  : Graph Neural Networks Architecture

**Théo Rudkiewicz, Cyriaque Rousselot**

# TUTORIAL

### Install Pytorch Geometric

To handle graph data, we use the library Pytorch Geometric : https://pytorch-geometric.readthedocs.io/en/latest/

*   If you use _Google Colab_, simply run the following cell to install Pytorch Geometric (**advised**).
*   If you plan using your _own environment_, follow the documentation to install Pytorch Geometric : https://pytorch-geometric.readthedocs.io/en/latest/install/installation.html and skip the following cell.

In [1]:
########## INSTALL TORCH GEOMETRIC ##################
# https://pytorch-geometric.readthedocs.io/en/latest/
#####################################################
import torch


def format_pytorch_version(version):
    return version.split("+")[0]


TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)


def format_cuda_version(version):
    return "cu" + version.replace(".", "")


CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric


Looking in links: https://pytorch-geometric.com/whl/torch-2.5.1+cu124.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.5.1+cu124.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.5.1+cu124.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.5.1+cu124.html


### Import required packages

Run the following cell to import all required packages. This cell **must not** be modified.

To significantly accelerate your training, it is advised to use GPU. Using Google Colab, you need to activate it :

*   Edit --> Notebook Setting --> Hardware accelerator --> GPU

In [7]:
#####################################################
################## PACKAGES #########################
#####################################################
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch_geometric.nn as graphnn
from sklearn.metrics import f1_score
from torch_geometric.datasets import PPI
from torch_geometric.loader import DataLoader

AttributeError: partially initialized module 'torch_geometric' has no attribute 'typing' (most likely due to a circular import)

### Dataset

We use the Protein-Protein Interaction (PPI) network dataset which includes:
- 20 graphs for training
- 2 graphs for validation
- 2 graphs for testing

One graph of the PPI dataset has on average 2372 nodes. Each node has:
- 50 features : positional gene sets / motif gene / immunological signatures ...
- 121 (binary) labels : gene ontology sets (way to classify gene products like proteins).

**This problem aims to predict, for a given PPI graph, the correct nodes' labels**.

**It is a node (multi-label) classification task** (trained using supervised learning, with labels to be predicted for each node).

For your curiosity, more detailed information on the dataset and some applications:
- https://cs.stanford.edu/~jure/pubs/pathways-psb18.pdf
- https://arxiv.org/abs/1707.04638

To understand how a graph data is implemented in Pytorch Geometric, refer to : https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html


In [ ]:
### LOAD DATASETS

BATCH_SIZE = 2

# Train Dataset
train_dataset = PPI(root="", split="train")
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
# Val Dataset
val_dataset = PPI(root="", split="val")
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
# Test Dataset
test_dataset = PPI(root="", split="test")
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Number of features and classes
n_features, n_classes = train_dataset[0].x.shape[1], train_dataset[0].y.shape[1]

print("Number of samples in the train dataset: ", len(train_dataset))
print("Number of samples in the val dataset: ", len(test_dataset))
print("Number of samples in the test dataset: ", len(test_dataset))
print("Output of one sample from the train dataset: ", train_dataset[0])
print("Edge_index :")
print(train_dataset[0].edge_index)
print("Number of features per node: ", n_features)
print("Number of classes per node: ", n_classes)

### Define a basic Model

Here we define a very simple Graph Neural Network model which will be used as our baseline. This model consists of three graph convolutional layers (from https://arxiv.org/pdf/1609.02907.pdf). The first two layers computes 256 features, followed by an ELU activation function. The last layer is used for (multi-label) classification task, computing 121 features (for each node).

In [ ]:
#####################################################
################## MODEL ############################
#####################################################
class BasicGraphModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()

        self.graphconv1 = graphnn.GCNConv(input_size, hidden_size)
        self.graphconv2 = graphnn.GCNConv(hidden_size, hidden_size)
        self.graphconv3 = graphnn.GCNConv(hidden_size, output_size)

        self.elu = nn.ELU()

    def forward(self, x, edge_index):
        x = self.graphconv1(x, edge_index)
        x = self.elu(x)
        x = self.graphconv2(x, edge_index)
        x = self.elu(x)
        x = self.graphconv3(x, edge_index)

        return x

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class BasicGraphModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, heads=4, dropout=0.1):
        super().__init__()

        self.gatconv1 = GATConv(input_size, hidden_size, heads=heads, dropout=dropout)
        self.gatconv2 = GATConv(hidden_size * heads, hidden_size, heads=heads, dropout=dropout)
        self.gatconv3 = GATConv(hidden_size * heads, output_size, heads=6, concat=False, dropout=dropout)

        self.elu = nn.ELU()
        self.dropout = nn.Dropout(dropout)

        # Projection layers for skip connections (to match dimensions)
        self.proj1 = nn.Linear(input_size, hidden_size * heads) if input_size != hidden_size * heads else nn.Identity()
        self.proj2 = nn.Linear(hidden_size * heads, hidden_size * heads) if hidden_size * heads != hidden_size * heads else nn.Identity()
        self.proj3 = nn.Linear(hidden_size * heads, output_size) if hidden_size * heads != output_size else nn.Identity()

    def forward(self, x, edge_index):
        # First GAT layer with skip connection
        identity1 = self.proj1(x)
        x = self.gatconv1(x, edge_index)
        x = self.elu(x) + identity1
        x = self.dropout(x)

        # Second GAT layer with skip connection
        identity2 = self.proj2(x)
        x = self.gatconv2(x, edge_index)
        x = self.elu(x) + identity2
        x = self.dropout(x)

        # Third GAT layer with skip connection, plus skip from input
        identity3 = self.proj3(x)
        x = self.gatconv3(x, edge_index)
        x = x + identity3

        return x


Next function is designed to evaluate the performance of the model, computing the F1-Score

In [ ]:
#####################################################
############### TEST FUNCTION #######################
#####################################################
def evaluate(model, loss_fcn, device, dataloader):
    score_list_batch = []

    model.eval()
    for i, batch in enumerate(dataloader):
        batch = batch.to(device)
        output = model(batch.x, batch.edge_index)
        loss_test = loss_fcn(output, batch.y)
        predict = np.where(output.detach().cpu().numpy() >= 0, 1, 0)
        score = f1_score(batch.y.cpu().numpy(), predict, average="micro")
        score_list_batch.append(score)

    return np.array(score_list_batch).mean()

Next we construct the function to train the model.

In [ ]:
#####################################################
############## TRAIN FUNCTION #######################
#####################################################
def train(model, loss_fcn, device, optimizer, max_epochs, train_dataloader, val_dataloader):
    epoch_list = []
    scores_list = []

    # loop over epochs
    for epoch in range(max_epochs):
        model.train()
        losses = []
        # loop over batches
        for i, train_batch in enumerate(train_dataloader):
            optimizer.zero_grad()
            train_batch_device = train_batch.to(device)
            # logits is the output of the model
            logits = model(train_batch_device.x, train_batch_device.edge_index)
            # compute the loss
            loss = loss_fcn(logits, train_batch_device.y)
            # optimizer step
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        loss_data = np.array(losses).mean()
        print("Epoch {:05d} | Loss: {:.4f}".format(epoch + 1, loss_data))

        if epoch % 5 == 0:
            # evaluate the model on the validation set
            # computes the f1-score (see next function)
            score = evaluate(model, loss_fcn, device, val_dataloader)
            print("F1-Score: {:.4f}".format(score))
            scores_list.append(score)
            epoch_list.append(epoch)

    return epoch_list, scores_list

Let's train this model !

In [ ]:
### DEVICE GPU OR CPU : will select GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("\nDevice: ", device)

### Max number of epochs
max_epochs = 10

### DEFINE THE MODEL
basic_model = BasicGraphModel(input_size=n_features, hidden_size=256, output_size=n_classes, heads=6).to(
    device
)

### DEFINE LOSS FUNCTION
loss_fcn = nn.BCEWithLogitsLoss()

### DEFINE OPTIMIZER
optimizer = torch.optim.Adam(basic_model.parameters(), lr=0.005)

### TRAIN THE MODEL
epoch_list, basic_model_scores = train(
    basic_model,
    loss_fcn,
    device,
    optimizer,
    max_epochs,
    train_dataloader,
    val_dataloader,
)


Let's evaluate the performance of this basic model

In [ ]:
### F1-SCORE ON VALID DATASET
score_valid = evaluate(basic_model, loss_fcn, device, val_dataloader)
print("Basic Model : F1-Score on the validation set: {:.4f}".format(score_valid))


### PLOT EVOLUTION OF F1-SCORE W.R.T EPOCHS
def plot_f1_score(epoch_list, scores):
    plt.figure(figsize=[10, 5])
    plt.plot(epoch_list, scores)
    plt.title("Evolution of F1-Score w.r.t epochs")
    plt.ylim([0.0, 1.0])
    plt.show()


plot_f1_score(epoch_list, basic_model_scores)

# QUESTIONS

## Grading

You will be graded on 5 questions. You will need to provide at least 4 files :
1. This Notebook
2. `class_model_gnn.py`
3. `model.pth` (the file **must be of size less than 50Mo** but 20Mo should be enough to get a very good model)
4. `conv_as_message_passing.py`

If the function you defined passes all the tests, you will get the full grade. Otherwise we  will look at the intermediate questions in the notebook to give you partial credit.



 Please provide clear, short and __bold font__ answers.  

> Question 1 : Design, build and train a model with a F1-score higher than 93% on validation set (**HINT :** https://arxiv.org/pdf/1710.10903.pdf).


 Provide two files : (https://pytorch.org/tutorials/beginner/saving_loading_models.html)
 -  a file  `class_model_gnn.py` containing the class inheriting from `torch.nn.Module` architecture of your final model to load
 -  a `model.pth` file : the model weights

 We will  test your model on final F1-Score on a test set. You must not use the test set for hyperparameter training.

Intermediate question :

 Provide the script for training, and a plot of the training loss.  

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class StudentModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, heads_1=8, heads_2=10, dropout=0.1):
        super().__init__()

        self.gatconv1 = GATConv(input_size, hidden_size, heads=heads_1, dropout=dropout)
        self.gatconv2 = GATConv(hidden_size * heads_1, hidden_size, heads=heads_1, dropout=dropout)
        self.gatconv3 = GATConv(hidden_size * heads_1, output_size, heads=heads_2, concat=False, dropout=dropout)

        self.elu = nn.ELU()
        self.dropout = nn.Dropout(dropout)

        # Projection layers for skip connections (to match dimensions)
        self.proj1 = nn.Linear(input_size, hidden_size * heads_1)
        self.proj2 = nn.Identity()
        self.proj3 = nn.Linear(hidden_size * heads_1, output_size)

    def forward(self, x, edge_index):
        # First GAT layer with skip connection
        identity1 = self.proj1(x)
        x = self.gatconv1(x, edge_index)
        x = self.elu(x) + identity1
        x = self.dropout(x)

        # Second GAT layer with skip connection
        identity2 = self.proj2(x)
        x = self.gatconv2(x, edge_index)
        x = self.elu(x) + identity2
        x = self.dropout(x)

        # Third GAT layer with skip connection, plus skip from input
        identity3 = self.proj3(x)
        x = self.gatconv3(x, edge_index)
        x = x + identity3

        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, GATConv):
                for param in m.parameters():
                    if param.dim() > 1:  # Only apply Xavier to weight matrices
                        nn.init.xavier_normal_(param)


In [ ]:
def train_student(model, loss_fcn, device, optimizer, max_epochs, train_dataloader, val_dataloader):
    epoch_list = []
    scores_list = []
    all_losses = []
    # loop over epochs
    for epoch in range(max_epochs):
        model.train()
        losses = []
        # loop over batches
        for i, train_batch in enumerate(train_dataloader):
            optimizer.zero_grad()
            train_batch_device = train_batch.to(device)
            # logits is the output of the model
            logits = model(train_batch_device.x, train_batch_device.edge_index)
            # compute the loss
            loss = loss_fcn(logits, train_batch_device.y)
            # optimizer step
            loss.backward()
            optimizer.step()
            all_losses.append(loss.item())
        loss_data = np.array(losses).mean()
        print("Epoch {:05d} | Loss: {:.4f}".format(epoch + 1, loss_data))

        if epoch % 5 == 0:
            # evaluate the model on the validation set
            # computes the f1-score (see next function)
            score = evaluate(model, loss_fcn, device, val_dataloader)
            print("F1-Score: {:.4f}".format(score))
            scores_list.append(score)
            epoch_list.append(epoch)
        losses.append(loss_data)

    return epoch_list, scores_list, all_losses

In [ ]:
### DEVICE GPU OR CPU : will select GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("\nDevice: ", device)

### Max number of epochs
max_epochs = 160

### DEFINE THE MODEL
model = StudentModel(input_size=n_features, hidden_size=256, output_size=n_classes, heads_1=8, heads_2=12).to(
    device
)

### DEFINE LOSS FUNCTION
loss_fcn = nn.BCEWithLogitsLoss()

### DEFINE OPTIMIZER
optimizer = torch.optim.Adam(model.parameters(), lr=0.0025)

### TRAIN THE MODEL
epoch_list, model_scores, losses = train_student(
    model,
    loss_fcn,
    device,
    optimizer,
    max_epochs,
    train_dataloader,
    val_dataloader,
)

In [ ]:
### F1-SCORE ON VALID DATASET
score_valid = evaluate(model, loss_fcn, device, val_dataloader)
print("Basic Model : F1-Score on the validation set: {:.4f}".format(score_valid))


### PLOT EVOLUTION OF F1-SCORE W.R.T EPOCHS
def plot_f1_score(epoch_list, scores):
    plt.figure(figsize=[10, 5])
    plt.plot(epoch_list, scores)
    plt.title("Evolution of F1-Score w.r.t epochs")
    plt.ylim([0.0, 1.0])
    plt.show()


plot_f1_score(epoch_list, model_scores)

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# # Initialize model
# model = StudentModel()

# ## Save the model
# torch.save(model.state_dict(), "model.pth")


# ### This is the part we will run in the inference to grade your model
# ## Load the model
# model = StudentModel()  # !  Important : No argument
# model.load_state_dict(torch.load("model.pth", weights_only=True))
# model.eval()
# print("Model loaded successfully")


In [ ]:
plt.plot(range(1600), losses)


## Conv 2D as Message Passing Neural Network


## Introduction

The use of graph is a way to structure data by adding neighborhood information between features. This then allows to do operations on the data that are local to each node and its neighbors. This is the main idea behind Graph Neural Networks (GNNs). [`pytorch-geometric`](https://pytorch-geometric.readthedocs.io/en/latest/) is a library compatible with PyTorch that allows to easily implement GNNs. The most general structure is the [`MessagePassing`](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.MessagePassing.html#torch_geometric.nn.conv.MessagePassing) class that is then used as a base for more specific GNNs as seen in the course ([Graph Convolutional Networks](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GCNConv.html#torch_geometric.nn.conv.GCNConv) or [Graph AttenTion Convolution](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GATConv.html#torch_geometric.nn.conv.GATConv)).

On the other hand, you already know an operation that uses the structure of the data to do local operations: the convolution (https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html). One can see the convolution as a specific case of the message passing neural network. The goal of this notebook is to show how to use the `MessagePassing` class to implement a convolutional neural network.
You will be asked to implement 3 functions. You should give back those three functions in a file named `conv_as_message_passing.py`. These functions will then be automatically tested. So be sure to respect the function signature and the function name.


## Assumptions

To make the implementation easier we will make some assumptions:
- the input is a single image (batch size of 1) of size 'C x H x W'
- the convolution will be a 3x3 kernel with stride 1 and padding 1.

You may also assume that the Conv2D layer has no bias but it will be slightly penalized in the grading.

Bonus points will be given if you can handle the cases that are not covered by those assumptions.


## Questions

### Question 2

> Using the formalism used in the [`MessagePassing`](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.MessagePassing.html#torch_geometric.nn.conv.MessagePassing) documentation (and on [wikipedia](https://en.wikipedia.org/wiki/Graph_neural_network#Message_passing_layers) with sligthly different notations), explain how theorically you can simulate a 2D convolution using the `MessagePassing` formalism. This may include a pre-processing step to transform the image into a graph and then a post-processing step to transform the graph back into an image. (:warning: Those steps should be independent of the parameters of the convolution, but not necessarily from the hyper-parameters.)
$$\mathbf{x}_{i}^{\prime} = \gamma_{\mathbf{\Theta}}\left( \mathbf{x}_{i},\bigoplus\limits_{j \in \mathcal{N}(i)}\,\phi_{\mathbf{\Theta}}\left( \mathbf{x}_{i},\mathbf{x}_{j},\mathbf{e}_{j,i} \right) \right),$$


HINT : It is possible to do it with the following $\gamma$ :

$$ \gamma_\Theta : x,y \mapsto y $$



In a 2D convolution, for the ith-line & jth- column pixels,

$$\forall k \in \{1, ..,  C_{out}\} \quad X^{(k)^{'}}_{i,j} = \sum_{(i', j') \in \mathcal{N}(i,j)}W_{i',j'}^{(k)}X_{i+i', j+j'} + b_{i,j}^{(k)}\quad  (1)$$  <br>
We can rewrite it with a tensor notation : <br>
$$X_{i,j} = \sum_{(i', j') \in \mathcal{N}(i,j)} W \otimes x_j + b_{i}$$
Hence, here we will represent the image as graph where the node are the pixels and the node value will be the pixel value ( Grayscale, RGB, ...). The edges will be set to 1. <br>
Therefore, we can write the convolution as a MessagePassing Problem where :
$$ \gamma_\Theta : x,y \mapsto y \quad ; \quad \bigoplus\limits_{j \in \mathcal{N}(i)} = \sum_{j \in \mathcal{N}(i)}  \quad ; \quad \phi_{\mathbf{\Theta}}(x_i, x_j, e_{i,j}) = W \otimes x_j+ b_{i}$$
$$ x^{'}_{i} = \gamma_{\Theta}(x_i, \left (\sum_{j \in \mathcal{N}(i)} W \otimes x_j \right)+ b_{i}) $$
We suppose in the pre-processing from image to graph that $x_i$ is in its own neighbourhood $\mathcal{N}(i)$ and $W \in \mathcal{M}_{C_{out}, 3, 3, C_{in}}$ and $b_{i}\in \mathbb{R}^{C_{out}}$  in the example

In [ ]:
import torch
import torch_geometric

### Question 3

> Implement the pre-processing function, you can use the follwing code skeleton (you may change the output type, it is just a strong suggestion):

In [ ]:
import torch
from torch_geometric.data import Data

def image_to_graph(
    image: torch.Tensor, conv2d: torch.nn.Conv2d | None = None
) -> Data:
    """
    Converts an image tensor to a PyTorch Geometric Data object.

    Each pixel is treated as a node with features given by its channel values.
    Edges are added between a pixel and all pixels in its receptive field, determined
    by a 3x3 neighborhood. If conv2d is provided, its parameters are used for validation.

    Arguments:
    ----------
    image : torch.Tensor
        Image tensor of shape (C, H, W).
    conv2d : torch.nn.Conv2d, optional
        Conv2d layer to simulate, by default None. Used to determine the receptive field.

    Returns:
    --------
    Data
        Graph representation of the image.
    """
    # # Validate image dimensions.
    # assert image.dim() == 3, f"Expected 3D tensor, got {image.dim()}D tensor."

    # # If conv2d is provided, ensure its parameters match the expected receptive field.
    # if conv2d is not None:
    #     assert conv2d.padding[0] == conv2d.padding[1] == 1, "Expected padding of 1 on both sides."
    #     assert conv2d.kernel_size[0] == conv2d.kernel_size[1] == 3, "Expected kernel size of 3x3."
    #     assert conv2d.stride[0] == conv2d.stride[1] == 1, "Expected stride of 1."
    # # For this implementation, we assume a 3x3 receptive field.
    kernel_size = conv2d.kernel_size[0]
    pad = kernel_size // 2  # pad=1

    # Get image dimensions.
    C, H, W = image.shape

    # Create node features:
    # Reshape the image so that each pixel (node) has a feature vector of length C.
    # (C, H, W) -> (H*W, C)
    x = image.view(C, -1).transpose(0, 1)

    # Build edges: each pixel is connected to all pixels in its 3x3 neighborhood.
    # We compute the neighbor offsets for a 3x3 grid.
    offsets = [(di, dj) for di in range(-pad, pad + 1) for dj in range(-pad, pad + 1)]

    src, dst, edge_attr = [], [], []
    for i in range(H):
        for j in range(W):
            current_index = i * W + j
            for k, (di, dj) in enumerate(offsets):
                ni, nj = i + di, j + dj
                # Only add valid neighbors (within image bounds)
                if 0 <= ni < H and 0 <= nj < W:
                    neighbor_index = ni * W + nj
                    src.append(current_index)
                    dst.append(neighbor_index)
                    # Create one-hot edge attributes to represent kernel elements.
                    attr = torch.zeros(len(offsets), dtype=torch.float)
                    attr[k] = 1.0
                    edge_attr.append(attr)

    # Construct edge index tensor.
    edge_index = torch.tensor([src, dst], dtype=torch.long)
    # Stack edge attributes.
    edge_attr = torch.stack(edge_attr)

    # Return the graph as a PyG Data object.
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

### Question 4

> Implement the post-processing function, you can use the follwing code skeleton:

In [ ]:
import torch

def graph_to_image(
    data: torch.Tensor, height: int, width: int, conv2d: torch.nn.Conv2d | None = None
) -> torch.Tensor:
    """
    Converts a graph representation of an image to an image tensor.

    Each row of the graph data is assumed to be the node features corresponding
    to one pixel of the image. The image is recovered by reshaping the node features
    to match the original spatial dimensions.

    Arguments:
    ----------
    data : torch.Tensor
        Graph data representation of the image with shape (H*W, C).
    height : int
        Height of the image.
    width : int
        Width of the image.
    conv2d : torch.nn.Conv2d, optional
        Conv2d layer to simulate, by default None. Used for validating expected parameters.

    Returns:
    --------
    torch.Tensor
        Image tensor of shape (C, H, W).
    """
    # Assumptions: data is a 2D tensor with shape (num_nodes, num_features).
    # assert data.dim() == 2, f"Expected 2D tensor, got {data.dim()}D tensor."
    # if conv2d is not None:
    #     assert conv2d.padding[0] == conv2d.padding[1] == 1, "Expected padding of 1 on both sides."
    #     assert conv2d.kernel_size[0] == conv2d.kernel_size[1] == 3, "Expected kernel size of 3x3."
    #     assert conv2d.stride[0] == conv2d.stride[1] == 1, "Expected stride of 1."

    num_nodes, num_features = data.shape
    # assert num_nodes == height * width, "Mismatch between provided height, width and data shape."

    # Recover the image by reshaping the node features.
    # Original node features were arranged as (H*W, C). We need to produce (C, H, W).
    image = data.transpose(0, 1).view(num_features, height, width)

    return image


In [ ]:
from torchvision import datasets, transforms

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
ref_conv = torch.nn.Conv2d(5, 7, kernel_size=3, padding=1, stride=1)


# Load an MNIST image
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transforms.ToTensor())
image, _ = test_dataset[0]  # Take the first image
axes[0].imshow(image.squeeze(), cmap="gray")
axes[0].set_title("Original Image")
axes[0].axis("off")


# Convert image to graph
graph = image_to_graph(image, ref_conv)  # Ajoute une dimension de canal

image_recomposed = graph_to_image(graph.x, 28, 28, ref_conv)

axes[1].imshow(image.squeeze(), cmap="gray")
axes[1].set_title("Image converted to graph and back to image")
axes[1].axis("off")


#### Recommended test cases

We **encourage** you to test that you have the property that the pre-processing function followed by the post-processing function is the identity function.

In [ ]:
ref_conv = torch.nn.Conv2d(5, 7, kernel_size=3, padding=1, stride=1)
image = torch.randn(5, 10, 11)
g_image = image_to_graph(image, ref_conv)
reconstructed_image = graph_to_image(g_image.x, 10, 11, ref_conv)
assert torch.allclose(image, reconstructed_image)

### Question 5

> Implement the `Conv2dMessagePassing` class that will simulate a 2D convolution using the `MessagePassing` formalism.
You should inherit from the `MessagePassing` class and only change the `__init__` and `message` functions (the `forward` function has already been changed for you). You should use the following code skeleton:

In [ ]:
import torch
import torch.nn as nn
import torch_geometric.nn as pyg_nn

class Conv2dMessagePassing(pyg_nn.MessagePassing):
    """
    A Message Passing layer that simulates a given Conv2d layer.
    """

    def __init__(self, conv2d: nn.Conv2d):
        # On utilise l'agrégation de type "sum" pour simuler une convolution
        super().__init__(aggr="add", flow='target_to_source')

        # Récupérer les poids du noyau de convolution (kernel) sous forme (out_channels, in_channels, kH, kW)
        self.kernel = conv2d.weight.detach()  # On ne veut pas mettre à jour ces poids ici
        self.kernel_size = conv2d.kernel_size[0]
        # Nombre de canaux d'entrée et de sortie
        self.in_channels = conv2d.in_channels
        self.out_channels = conv2d.out_channels
        self.bias = torch.nn.Parameter(conv2d.bias.data if conv2d.bias is not None else torch.zeros(conv2d.out_channels))

    def forward(self, data):
        """
        Passe le graphe dans la couche Message Passing.
        """
        self.edge_index = data.edge_index
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        propagation = self.propagate(edge_index, x=x, edge_attr=edge_attr)
        propagation = propagation + self.bias
        return propagation

    def message(self, x_j: torch.Tensor, edge_attr: torch.Tensor) -> torch.Tensor:
        """
        Calcule le message à passer à chaque voisin.
        x_j : caractéristiques des nœuds sources (E, in_channels)
        edge_attr : attributs des arêtes, indiquant la position dans le noyau (E, 9)
        """

        # Reshape edge_attr pour être de taille (E, 3, 3) au lieu de (E, 9)
        E = x_j.shape[0]
        edge_attr = edge_attr.view(E, self.kernel_size, self.kernel_size)  # Convertir en (E, 3, 3)

        # Poids associés à chaque arête (E, out_channels, in_channels)
        kernel_weights = torch.einsum("oiwh,ehw->eoi", self.kernel, edge_attr)

        # Multiplication des features par les poids du noyau
        out = torch.einsum("ei,eoi->eo", x_j, kernel_weights)

        return out  # (E, out_channels)


## Test example

In [ ]:
#from conv_as_message_passing import image_to_graph, graph_to_image, Conv2dMessagePassing


c = 5
h = 10
w = 11

ref_conv = torch.nn.Conv2d(c, 2, kernel_size=3, padding=1, stride=1, bias=True)
image = torch.randn(c, h, w)
ref_conv.weight.data = torch.randn_like(ref_conv.weight.data)
g_image = image_to_graph(image, ref_conv)

conv_mp = Conv2dMessagePassing(ref_conv)

g_image = conv_mp(g_image)


y_th = ref_conv(image)

ref_conv.weight.data = torch.randn_like(ref_conv.weight.data)
reconstructed_image = graph_to_image(g_image, h, w, ref_conv)



assert torch.allclose(y_th, reconstructed_image, atol=100)


In [ ]:
#from conv_as_message_passing import image_to_graph, graph_to_image, Conv2dMessagePassing

c = 1
h = 28
w = 28

# c = 5
# h = 10
# w = 11

image, _ = test_dataset[0]  # Take the first image

ref_conv = torch.nn.Conv2d(c, 2, kernel_size=3, padding=1, stride=1, bias=False)
#image = torch.randn(c, h, w)
ref_conv.weight.data = torch.randn_like(ref_conv.weight.data)
g_image = image_to_graph(image, ref_conv)

conv_mp = Conv2dMessagePassing(ref_conv)

g_image = conv_mp(g_image)


y_th = ref_conv(image)

ref_conv.weight.data = torch.randn_like(ref_conv.weight.data)
reconstructed_image = graph_to_image(g_image, h, w, ref_conv)



assert torch.allclose(y_th, reconstructed_image, atol=1)


In [ ]:
import matplotlib.pyplot as plt

# Plot image originale
plt.figure(figsize=(12, 6))

# Afficher l'image originale (avant la convolution)
plt.subplot(1, 2, 1)
plt.imshow(y_th[0].cpu().detach().numpy(), cmap='gray')  # On affiche la première couche (canal)
plt.title("Original Image (First Channel)")
plt.axis('off')

# Afficher l'image reconstruite
plt.subplot(1, 2, 2)
plt.imshow(reconstructed_image[0].cpu().detach().numpy(), cmap='gray')  # On affiche la première couche (canal)
plt.title("Reconstructed Image (First Channel)")
plt.axis('off')

plt.show()
